In [1]:
import concurrent.futures
import jinja2
from  anthropic import AnthropicBedrock
from compiler.core import Compiler
from tracing_client import TracingClient
from policies import common, handlers, typespec, drizzle, typescript, router

In [2]:
jinja_env = jinja2.Environment()
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
client = TracingClient(AnthropicBedrock(aws_profile="dev", aws_region="us-west-2"))

In [3]:
application_description = "Bot that writes poems about cars."

In [4]:
import os

DATASET_DIR = "evals/dataset.min/"
files = os.listdir(DATASET_DIR)
#typespec_definitions = "car_poem_typespec_schema.ts"
typescript_file = "car_poem_typescript_schema.ts"
drizzle_file = "car_poem_drizzle_schema.ts"
typespec_definitions = ""
#with open(os.path.join(DATASET_DIR, typespec_definitions), "r", encoding="utf-8") as f:
#    typespec_definitions = f.read()
with open(os.path.join(DATASET_DIR, typescript_file), "r", encoding="utf-8") as f:
    typescript_definitions = f.read()
with open(os.path.join(DATASET_DIR, drizzle_file), "r", encoding="utf-8") as f:
    drizzle_schema = f.read()

content = jinja_env.from_string(handlers.PROMPT).render(application_description=application_description,
                                                        function_name="generateCarPoem",
                                                        typespec_schema=typespec_definitions,
                                                        typescript_schema=typescript_definitions,
                                                        drizzle_schema=drizzle_schema)
message = {"role": "user", "content": content}

In [5]:
message

{'role': 'user',
 'content': 'Based on TypeScript application definition and drizzle schema, generate a handler for getCarPoem function.\nHandler always accepts single argument. It should be declared at the beginning as interface Options;\nHandler should satisfy following interface:\n\n<handler>\ninterface Message {\n    role: \'user\' | \'assistant\';\n    content: string;\n};\n\ninterface Handler<Options, Output> {\n    preProcessor: (input: Message[]) => Options | Promise<Options>;\n    handle: (options: Options) => Output | Promise<Output>;\n    postProcessor: (output: Output, input: Message[]) => Message[] | Promise<Message[]>;\n}\n\nclass GenericHandler<Options, Output> implements Handler<Options, Output> {\n    constructor(\n        public handle: (options: Options) => Output | Promise<Output>,\n        public preProcessor: (input: Message[]) => Options | Promise<Options>,\n        public postProcessor: (output: Output, input: Message[]) => Message[] | Promise<Message[]>\n    ) 

In [6]:
with handlers.HandlerTaskNode.platform(client, compiler, jinja_env):
    output = handlers.HandlerTaskNode.run([message], function_name="generateCarPoem",
                                          typespec_schema=typespec_definitions,
                                          typescript_schema=typescript_definitions,
                                          drizzle_schema=drizzle_schema)

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


In [7]:
output.output


HandlerOutput(handler='import { db } from "../db";\nimport { carPoemsTable, poemStylesTable } from "../db/schema/application";\nimport { eq } from "drizzle-orm";\nimport { GenericHandler, type Message } from "../common/handler";\nimport { CarPoem, CarPoemBot } from "../common/schema";\n\ninterface Options {\n    id: number;\n}\n\nasync function handle(options: Options): Promise<CarPoem> {\n    const poem = await db.select({\n        title: carPoemsTable.title,\n        content: carPoemsTable.content,\n        topic: carPoemsTable.topic,\n        created_at: carPoemsTable.created_at,\n        style_name: poemStylesTable.name,\n        verses_count: poemStylesTable.verses_count,\n        lines_per_verse: poemStylesTable.lines_per_verse\n    })\n    .from(carPoemsTable)\n    .innerJoin(poemStylesTable, eq(carPoemsTable.style_id, poemStylesTable.id))\n    .where(eq(carPoemsTable.id, options.id))\n    .execute();\n\n    if (!poem.length) {\n        throw new Error(\'Poem not found\');\n    

In [8]:
root = handlers.HandlerTaskNode(output)

In [9]:
with handlers.HandlerTaskNode.platform(client, compiler, jinja_env):
    solution = common.bfs(root)

In [10]:
solution.score, solution.depth

(0, 0)

In [11]:
if solution.parent:
    for node in solution.parent.children:
      print(node.is_successful, node.data.output)

In [12]:
with handlers.HandlerTaskNode.platform(client, compiler, jinja_env):
    args = root.run_args

In [13]:
args

[{'role': 'assistant',
  'content': 'Based on the provided schemas and definitions, I\'ll generate a handler for getCarPoem function. The handler will need to interact with poem-related tables and return a CarPoem object.\n\n<handler>\nimport { db } from "../db";\nimport { carPoemsTable, poemStylesTable } from "../db/schema/application";\nimport { eq } from "drizzle-orm";\nimport { GenericHandler, type Message } from "../common/handler";\nimport { CarPoem, CarPoemBot } from "../common/schema";\n\ninterface Options {\n    id: number;\n}\n\nasync function handle(options: Options): Promise<CarPoem> {\n    const poem = await db.select({\n        title: carPoemsTable.title,\n        content: carPoemsTable.content,\n        topic: carPoemsTable.topic,\n        created_at: carPoemsTable.created_at,\n        style_name: poemStylesTable.name,\n        verses_count: poemStylesTable.verses_count,\n        lines_per_verse: poemStylesTable.lines_per_verse\n    })\n    .from(carPoemsTable)\n    .inn

# End to end test

In [4]:
application_description = "Bot that writes poems about cars."

content = jinja_env.from_string(typespec.PROMPT).render(application_description=application_description)
message = {"role": "user", "content": content}
with typespec.TypespecTaskNode.platform(client, compiler, jinja_env):
    tsp_data = typespec.TypespecTaskNode.run([message])
    tsp_root = typespec.TypespecTaskNode(tsp_data)
    tsp_solution = common.bfs(tsp_root)

match tsp_solution.is_successful:
    case True:
        print("Typespec solution is successful.")
    case False:
        raise Exception("Typespec solution is not successful.")

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


Typespec solution is successful.


In [16]:
content = jinja_env.from_string(router.PROMPT).render(typespec_definitions=tsp_solution.data.output.typespec_definitions)
message = {"role": "user", "content": content}
with router.RouterTaskNode.platform(client, jinja_env):
    router_data = router.RouterTaskNode.run([message], typespec_definitions=tsp_solution.data.output.typespec_definitions)
    router_root = router.RouterTaskNode(router_data)
    router_solution = common.bfs(router_root)

match router_solution.is_successful:
    case True:
        print("Router solution is successful.")
    case False:
        raise Exception("Router solution is not successful.")

Router solution is successful.


In [6]:
content = jinja_env.from_string(typescript.PROMPT).render(typespec_definitions=tsp_solution.data.output.typespec_definitions)
message = {"role": "user", "content": content}
with typescript.TypescriptTaskNode.platform(client, compiler, jinja_env):
    ts_data = typescript.TypescriptTaskNode.run([message])
    ts_root = typescript.TypescriptTaskNode(ts_data)
    ts_solution = common.bfs(ts_root)

match ts_solution.is_successful:
    case True:
        print("Typescript solution is successful.")
    case False:
        raise Exception("Typescript solution is not successful.")

Typescript solution is successful.


In [7]:
content = jinja_env.from_string(drizzle.PROMPT).render(typespec_definitions=tsp_solution.data.output.typespec_definitions)
message = {"role": "user", "content": content}
with drizzle.DrizzleTaskNode.platform(client, compiler, jinja_env):
    dzl_data = drizzle.DrizzleTaskNode.run([message])
    dzl_root = drizzle.DrizzleTaskNode(dzl_data)
    dzl_solution = common.bfs(dzl_root)

match dzl_solution.is_successful:
    case True:
        print("Drizzle solution is successful.")
    case False:
        raise Exception("Drizzle solution is not successful.")

Drizzle solution is successful.


In [13]:
dzl_solution.data.output.feedback

{'exit_code': 0,
 'stdout': "No config path provided, using default 'drizzle.config.ts'\nReading config file '/app/drizzle.config.ts'\nUsing 'pg' driver for database querying\n[⣷] Pulling schema from database...\n\x1b[2K\x1b[1G[✓] Pulling schema from database...\n[✓] Changes applied\n",
 'stderr': None}

In [15]:
print(dzl_solution.data.output.drizzle_schema)

import { index, integer, pgTable, text, timestamp, uuid } from "drizzle-orm/pg-core";

export const carsTable = pgTable("cars", {
  id: uuid("id").primaryKey().defaultRandom(),
  make: text("make").notNull(),
  model: text("model").notNull(),
  year: integer("year").notNull(),
});

export const poemsTable = pgTable("poems", {
  id: uuid("id").primaryKey().defaultRandom(),
  content: text("content").notNull(),
  car_id: uuid("car_id")
    .notNull()
    .references(() => carsTable.id),
  created_at: timestamp("created_at").notNull().defaultNow(),
});

export const favoritePoemsTable = pgTable("favorite_poems", {
  poem_id: uuid("poem_id")
    .primaryKey()
    .references(() => poemsTable.id),
  marked_at: timestamp("marked_at").notNull().defaultNow(),
});

// Indexes for better query performance
export const carSearchIndex = index("car_search_idx", carsTable, ["make", "model", "year"]);
export const poemDateIndex = index("poem_date_idx", poemsTable, ["created_at"]);
export const favori

In [8]:
def seach_handler(
    function_name: str,
    typespec_definitions: str,
    typescript_schema: str,
    drizzle_schema: str,
) -> handlers.HandlerTaskNode:
    prompt_params = {
        "application_description": application_description,
        "function_name": function_name,
        "typespec_schema": typespec_definitions,
        "typescript_schema": typescript_schema,
        "drizzle_schema": drizzle_schema,
    }
    content = jinja_env.from_string(handlers.PROMPT).render(**prompt_params)
    message = {"role": "user", "content": content}
    output = handlers.HandlerTaskNode.run([message], **prompt_params)
    root_node = handlers.HandlerTaskNode(output)
    solution = common.bfs(root_node)
    return solution

In [9]:
MAX_WORKERS = 5
results: dict[str, handlers.HandlerOutput] = {}
with handlers.HandlerTaskNode.platform(client, compiler, jinja_env):
    with concurrent.futures.ThreadPoolExecutor(MAX_WORKERS) as executor:
        future_to_handler = {}
        for function_name in tsp_solution.data.output.llm_functions:
            future_to_handler[executor.submit(
                seach_handler,
                function_name,
                tsp_solution.data.output.typespec_definitions,
                ts_solution.data.output.typescript_schema,
                dzl_solution.data.output.drizzle_schema,
            )] = function_name
        for future in concurrent.futures.as_completed(future_to_handler):
            function_name, result = future_to_handler[future], future.result()
            results[function_name] = result

In [10]:
for key, val in results.items():
    print("Handler", key)
    print(val.data.output.feedback["stdout"])
    print(val.depth)

Handler generatePoem
src/db/schema/application.ts(27,55): error TS2554: Expected 0-1 arguments, but got 3.
src/db/schema/application.ts(28,53): error TS2554: Expected 0-1 arguments, but got 3.
src/db/schema/application.ts(29,61): error TS2554: Expected 0-1 arguments, but got 3.

0
Handler saveFavoritePoem
src/db/schema/application.ts(27,55): error TS2554: Expected 0-1 arguments, but got 3.
src/db/schema/application.ts(28,53): error TS2554: Expected 0-1 arguments, but got 3.
src/db/schema/application.ts(29,61): error TS2554: Expected 0-1 arguments, but got 3.

0
Handler listFavoritePoems
src/db/schema/application.ts(27,55): error TS2554: Expected 0-1 arguments, but got 3.
src/db/schema/application.ts(28,53): error TS2554: Expected 0-1 arguments, but got 3.
src/db/schema/application.ts(29,61): error TS2554: Expected 0-1 arguments, but got 3.
src/handlers/listFavoritePoems.ts(22,46): error TS2339: Property 'equals' does not exist on type 'PgColumn<{ name: "id"; tableName: "poems"; dataTyp

In [11]:
print(ts_data.output.typescript_schema)

export interface CarDetails {
    make: string;
    carModel: string;
    year: number;
}

export interface PoemRequest {
    car: CarDetails;
    style: string;
    mood: string;
}

export interface Poem {
    id: string;
    content: string;
    carDetails: CarDetails;
    createdAt: Date;
}

export interface CarPoetBot {
    generatePoem(request: PoemRequest): Promise<Poem>;
    saveFavoritePoem(poem: Poem): Promise<void>;
    listFavoritePoems(from: Date, to: Date): Promise<Poem[]>;
}


In [14]:
print(results["generateCarPoem"].data.output.handler)

import { db } from "../db";
import { poemRequestsTable, poemsTable } from "../db/schema/application";
import type { PoemRequest, Poem } from "../common/schema";

interface Options {
    carType: string;
    poemStyle: string;
    mood: string;
}

export const handle = async (options: Options): Promise<Poem> => {
    const [poemRequest] = await db.insert(poemRequestsTable)
        .values({
            carType: options.carType,
            poemStyle: options.poemStyle,
            mood: options.mood
        })
        .returning();

    const poem: Poem = {
        title: `${options.mood} ${options.carType}`,
        content: `Generated poem content about ${options.carType}`,
        style: options.poemStyle
    };

    await db.insert(poemsTable)
        .values({
            requestId: poemRequest.id,
            title: poem.title,
            content: poem.content,
            style: poem.style
        })
        .execute();

    return poem;
};


In [18]:
results["greet"].output.feedback

{'exit_code': 0, 'stdout': None, 'stderr': None}

In [28]:
content = jinja_env.from_string(handlers.PROMPT).render(
    typespec_schema=tsp_solution.data.output.typespec_definitions,
    typescript_schema=ts_solution.data.output.typescript_schema,
    drizzle_schema=dzl_solution.data.output.drizzle_schema
)
message = {"role": "user", "content": content}
with handlers.HandlerTaskNode.platform(client, compiler, jinja_env):
    output = handlers.HandlerTaskNode.run([message],
                                          function_name="generateGreeting",
                                          typespec_schema=tsp_solution.data.output.typespec_definitions,
                                          typescript_schema=ts_solution.data.output.typescript_schema,
                                          drizzle_schema=dzl_solution.data.output.drizzle_schema)

In [29]:
print(output.output.feedback)

{'exit_code': 0, 'stdout': None, 'stderr': None}


In [30]:
print(output.output.handler)

import { db } from "../db";
import type { GreetingParams } from "../common/schema";
import { greetingParamsTable, generatedGreetingsTable } from "../db/schema/application";

interface Options {
    params: GreetingParams;
}

interface Output {
    greetingId: string;
    greeting: string;
}

export const handle = async (options: Options): Promise<Output> => {
    const [paramsResult] = await db.insert(greetingParamsTable)
        .values({
            name: options.params.name,
            time_of_day: options.params.timeOfDay
        })
        .returning({ id: greetingParamsTable.id });

    const greeting = `Hello${options.params.name ? " " + options.params.name : ""}${options.params.timeOfDay ? ", good " + options.params.timeOfDay : ""}!`;

    const [greetingResult] = await db.insert(generatedGreetingsTable)
        .values({
            params_id: paramsResult.id,
            greeting_text: greeting
        })
        .returning({ id: generatedGreetingsTable.id });

    return {
